In [1]:
import dolfinx
import gmsh
import tqdm
import copy
import cmasher
import numpy as np
import scipy.optimize
import scipy.interpolate
from scipy.interpolate import griddata

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.tri as mtri
import matplotlib.colors

from mpi4py import MPI
from Source.Mesh import *
from Source.Parameters import *
from Source.EnzymeFluxOptimized import *

domain_parameters = DomainParameters(dim=3)
mesh = Mesh(
    MPI.COMM_SELF,
    domain_parameters
)
parameters = Parameters(c_out = 0.7, k = 0.0, M = 1.0, χ = -0.01)

flux = EnzymeFlux(mesh, parameters)

def Δμ(v, M=None, k=None):    
    flux.update(velocity=v, M=M)
    return flux.get_asymmetry()

def dΔμ_dv(v, M=None, k=None, dv=1e-3):   
    Δμ_2 = Δμ(v + 0.5*dv, M=M, k=k)
    Δμ_1 = Δμ(v - 0.5*dv)    
    return (Δμ_2 - Δμ_1) / dv

INFO:root:running build_ext
INFO:root:building 'libffcx_forms_40931fafd13b9ebb7ca7c8baec04b6a0783fc1c9' extension
INFO:root:clang -Wno-unused-result -Wsign-compare -Wunreachable-code -DNDEBUG -fwrapv -O2 -Wall -fPIC -O2 -isystem /opt/homebrew/Caskroom/miniconda/base/envs/research/include -arch arm64 -fPIC -O2 -isystem /opt/homebrew/Caskroom/miniconda/base/envs/research/include -arch arm64 -I/opt/homebrew/Caskroom/miniconda/base/envs/research/lib/python3.10/site-packages/ffcx/codegeneration -I/opt/homebrew/Caskroom/miniconda/base/envs/research/include/python3.10 -c libffcx_forms_40931fafd13b9ebb7ca7c8baec04b6a0783fc1c9.c -o ./libffcx_forms_40931fafd13b9ebb7ca7c8baec04b6a0783fc1c9.o -O3
INFO:root:clang -bundle -undefined dynamic_lookup -Wl,-rpath,/opt/homebrew/Caskroom/miniconda/base/envs/research/lib -L/opt/homebrew/Caskroom/miniconda/base/envs/research/lib -Wl,-rpath,/opt/homebrew/Caskroom/miniconda/base/envs/research/lib -L/opt/homebrew/Caskroom/miniconda/base/envs/research/lib ./libf

In [2]:
# assign values to m
x0 = -5
L_grad   = 5
x = flux.m.function_space.tabulate_dof_coordinates()
r = np.linalg.norm(x - [x0, 0, 0], axis=1)
flux.m.x.array[:] = np.exp(-(r/L_grad)**2 ) / (1+(r/L_grad))

In [3]:
v = scipy.optimize.newton(lambda x: Δμ(x, M=1000), 0.0)

In [4]:
v

8.595780275100509

In [ ]:
Δμ(0.0, M=None, k=None)

In [ ]:
x0 = -5
# assign values to m
x = simulation.solution.function_space.tabulate_dof_coordinates()
r = np.linalg.norm(x - [x0, 0, 0], axis=1)
#simulation.m.x.array[:] = np.exp(-(r/L_grad)**2 ) / (1+(r/L_grad))


In [ ]:
simulation.solution

In [ ]:
k_range = np.linspace(0.1,10,100)
M_range = np.linspace(2000,1,100)

data_contour = []
data_bulk = []
for k in tqdm.tqdm(k_range):
    simulation.update(velocity=0, k=k)

    # find first value of M where Δμ starts to grow
    M_crit = scipy.optimize.newton(lambda x: dΔμ_dv(0.0, M=x), M_range.min())
    data_contour.append([k, M_crit])
    
    v = 10.0
    for M in M_range:
        if M >= M_crit:
            reference = Δμ(0.0, M=M) # this should ideally be zero, but there can be numerical inaccuracies
            v = scipy.optimize.newton(lambda x: Δμ(x, M=M) - reference, v)
            data_bulk.append([k, M, v, flux.get_asymmetry()])
        else:
            data_bulk.append([k, M, 0.0, flux.get_asymmetry()])

data_contour = np.array(data_contour)
data_bulk = np.array(data_bulk)

np.savetxt('droplet_motion_'+str(mesh.dimension)+'d_c'+str(parameters.c_out)+'_contour.csv', data_contour, delimiter=',', header="k,M_crit")
np.savetxt('droplet_motion_'+str(mesh.dimension)+'d_c'+str(parameters.c_out)+'_bulk.csv', data_bulk, delimiter=',', header="k,M,v,imbalance")